# Berman Standard State Code Generator 

A notebook to generate spud files for Berman standard state models

Required system packages and initialization

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import time
sym.init_printing()

Required ENKI packages (using ThermoCodegen version of the coder package)

In [ ]:
from thermocodegen.coder import coder

Get the working version of ThermoCodegen

In [ ]:
import thermocodegen as tcg
tcg_version = tcg.__version__
print('Using ThermoCodegen v{}'.format(tcg_version))

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../endmembers'
try:
    os.mkdir(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v{}/examples/Systems/fo_fa/notebooks/Generate_berman_endmembers.ipynb'.format(tcg_version)

## Clean the Berman Data base for different models

* Here we'll just use  standard state models without lambda transitions or order-disorder terms


In [ ]:
df = pd.read_json('data/berman_1988.json')
df.columns

In [ ]:
# pull out column names required for just standard state model
cols_ss = ['Phase', 'Formula', 'sAbbrev', 'lAbbrev', 'H_TrPr', 'S_TrPr', 'V_TrPr',
           'k0', 'k1', 'k2', 'k3', 'v1', 'v2', 'v3', 'v4']
df = df[cols_ss]

Rename some columns and just extract rows for phases Forsterite and Fayalite

In [ ]:
phases = ['Fo', 'Fa']
df = df.rename(index=str,columns={'Phase':'name', 'Formula':'formula'})
df = df[df.sAbbrev.isin(phases)]
df

## Generate Sympy symbols and Coder parameters from the column headers

To construct our Gibbs free energy we need to create Sympy symbols and units for the parameters

* H_TrPr: Enthalpy at reference $T$ and $P$ in $J$
* S_TrPr: Entropy at reference T and P in $J/K$
* V_TrPr: Volume at reference T and P in $J/bar$
* k0-k3:  Coefficients in Berman heat capacity model with units [ 'J/K-m', 'J-K^(1/2)-m', 'J-K/m', 'J-K^2' ]
* v1-v4:  Coefficients in Berman Equation of state (V) with units [ '1/bar', '1/bar^2' , '1/K', '1/K^2']


In [ ]:
param_strings = df.columns[4:]
units = [ 'J', 'J/K', 'J/bar']+ [ 'J/K-m', 'J-K^(1/2)-m', 'J-K/m', 'J-K^2' ]+[ '1/bar', '1/bar^2' , '1/K', '1/K^2']
print('parameter\tunit')
for t in list(zip(param_strings, units)):
    print('{}\t\t {}'.format(t[0],t[1]))

We now construct coder parameters and sympy symbols from the parameter names and pass them to the local dictionary for use in other sympy expressions

In [ ]:
params = coder.set_coder_params(param_strings, units)
symbol_dict = coder.get_symbol_dict_from_params(params)

# load local dictionary
locals().update(symbol_dict)


## Create  model class

Here we will use the thermocodgen extension .from_type() which defaults to a Gibbs-Free Energy models G(T,P)

In [ ]:
model = coder.StdStateModel.from_type()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

## Standard State Potentials

### Define model Potentials for the Standard State Potentials

Here we construct the Gibbs free energy $G(T,P)$ from two parts.

#### (1) $c_P$ integrals
The isobaric heat capacity terms are parameterized following Berman (1988) as : 

$$
    c_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3 
$$

and the heat capacity contribution to the Gibbs free energy is given by 

$$
    G_{P_r} = H(T_r,P_r) + \int_{T_r}^T C_p dT - T\left [S(T_r,P_r) + \int_{T_r}^T \frac{c_P}{T} dT\right ]
$$

where the reference condition third law entropy, $S(T_r,P_r)$, and enthalpy of formation from the
elements, $H(T_r,P_r)$, constitute additional parameters

In [ ]:
# Heat Capacity
Cp_Pr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3
Cp_Pr

Define the heat capacity contribution to the Gibbs free energy ...

In [ ]:
G_Pr = H_TrPr + sym.integrate(Cp_Pr,(T,Tr,T)) - T*(S_TrPr + sym.integrate(Cp_Pr/T,(T,Tr,T)))
G_Pr

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space. Here we use the Berman model

$$
    V(T,P) = V_{T_rP_r}(1 + v_1(P-P_r) + v_2(P-P_r)^2 + v_3(T-T_r) + v_4(T - T_r)^2)
$$

and calculate the contribution of $G$ due to pressure

$$
    G = \int_{P_r}^{P} V(T,P) dP
$$

In [ ]:
G_PrToP = sym.integrate(V_TrPr*(1+v1*(P-Pr)+v2*(P-Pr)**2+v3*(T-Tr)+v4*(T-Tr)**2),(P,Pr,P))
G_PrToP

#### Define standard state G as sum of $C_p$ integrals and EOS integrals

In [ ]:
G = G_Pr + G_PrToP
G

### Add potential and parameters to the model

In [ ]:
model.add_potential_to_model('G',G, params)

Exam settable values for the model dictionary

In [ ]:
model.get_values()                       

### Write out spud files for Berman endmembers for Fo and Fa

Here we loop over the data frame and populate the model dictionary with parameters from each endmember and output each endmember model as an Spud `.emml` file 

In [ ]:
# loop over the rows of the dataframe
for i, row in df.iterrows():
    # create a dictionary from each row
    values_dict = row.to_dict()
    
    # clean up endmember names to make them coder compliant
    values_dict['name'] ='{}_berman'.format(values_dict['name'].title())
    values_dict['name'] = values_dict['name'].replace('-','_')
    print(values_dict['name'])
    
    # set the reference string
    values_dict['reference']=reference
    
    # update the coder model with specific values and write xml to the appropriate directory
    model.set_values(values_dict)
    model.to_xml(path=SPUD_DIR)
